# Introduction
## What is a qumode?

Bits are the world's simplest alphabet, allowing any piece of information to be represented using the characters 0 and 1. Qubits are an extension of the bit to quantum mechanics, allowing us to encode arbitrary quantum information with just the states $|0\rangle$ and $|1\rangle$. But what if intead of two characters, the alphabet were infinite?

A bosonic mode, or "qumode," is an object that can contain any number of identical particles, each having the same energy. This endows the qumode with an infinite "alphabet" of states identified by the number of photons corresponding to that state, i.e. $|0\rangle, |1\rangle, |2\rangle, |3\rangle,...$, all the way to infinity. These states are formally referred to as "Fock" states.

![A qubit has two distringuishable states, whereas an oscillator has infinitely many](./figures/energy_diagram.png)

The figure above shows the two mutually exclusive states of a qubit (a) compared with the infinitely many states of a qumode (b). The energies of the states of the qumode are evenly spaced because each particle contributes the same energy.

## A note on Dirac notation
You may be familiar with representing quantum states using matrices with complex coefficients, and if you are comfortable with Dirac notation you can skip this section.

Dirac notation is another convenient way to represent quantum states. Dirac notation denotes a state by placing a label for the state between $|$ and $\rangle$. Mutually exclusive states, such as states with different energies or a different number of photons, are assigned to orthogonal vectors. This allows us to easily go back and forth between matrix notation and Dirac notation. For example, using the mutually exclusive states $|0\rangle$ and $|1\rangle$, 
$$
\alpha |0\rangle + \beta |1 \rangle \ \dot{=} \ \alpha \begin{pmatrix} 1 \\ 0 \end{pmatrix} + \beta \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \begin{pmatrix} \alpha \\ \beta \end{pmatrix}
$$
The funny $\dot =$ sign is used to indicate that the choice of orthogonal vectors to assign to each state is arbitrary, so we could easily have made a different choice to assign $|0 \rangle$ to $\begin{pmatrix} 0 \\ 1\end{pmatrix}$ and $|1\rangle$ to $\begin{pmatrix} 1 \\ 0 \end{pmatrix}$.

Dirac notation becomes particularly important for describing qumodes. With an infinite number of mutually exclusive states, the vectors would be infinitely long! The most general state $|\psi\rangle$ of a qumode can be written in the number basis as
$$
|\psi\rangle = \sum_{n=0}^\infty a_n |n\rangle
$$
where $a_n$ are complex numbers.

## From qubits to qumodes
Most of the familiar axioms for qubits generalize directly to qumodes. These are, briefly,
1. Mutually exclusive states are orthogonal: if $m\neq n$, then $|n\rangle$ and $|m\rangle$ correspond to a mode with two different photon numbers, making these states mutually exclusive. This implies that the states are orthogonal, meaning that their *inner product* vanishes: $\langle m | n \rangle = 0$. 
2. The Borne Rule: If $|\psi\rangle$ is the state of the resonator and $\{|n\rangle\}$ is an orthonormal basis, then the probability of measuring $|\psi\rangle$ in the state $|n\rangle$ is
$$
p_{\psi}(n) = |\langle n | \psi \rangle|^2
$$
3. Normalization: The Borne rule implies that all physical quantum states should be normalized. In the case of a discrete basis like the Fock basis, this implies that $\langle n | m \rangle = \delta_{mn}$. Vectors $|x\rangle$ comprising a continuous basis generally do not represent physical states, and they are delta-function normalized, meaning that $\langle x | x' \rangle = \delta(x-x')$. Learn more about this in `operations.ipynb`.

## Importing Bosonic Qiskit
Bosonic Qiskit is C2QA project to extend the functionality of qiskit to simulating hybrid boson-qubit systems within QisKit.

In [2]:
# The C2QA pacakge is currently not published to PyPI.
# To use the package locally, add the C2QA repository's root folder to the system path
import os
import sys
module_path = "/home/ben/Documents/Repos/bosonic-qiskit"
sys.path.append(module_path)

import c2qa
import c2qa.util as util
import qiskit
import numpy as np
import matplotlib.pyplot as plt

### Creating a Qumode

Because Qiskit is basaed on two-level qubits, Bosonic Qiskit extends Qiskit by approximating the infinite states of a *qumode* by using multiple qubits. The `QumodeRegister` is an abstraction of the `QuantumRegister` object designed to represent qumodes. The `num_qubits_per_qumode` parameter specifies the $k$ qubits used to simulate the first $2^k$ levels of a qumode. 

The following piece of code creates a qumode using 4 qubits. Using too few qubits to simulate a qumode may result in an innacurate simulation. It is up to the user to identify the number with reasonable performance while not sacrificing accuracy, and this can be done by making small adjustments to the parameter and checking for a significant change in the result. 4 qubits is a good starting point, but the runtime of simulations increases exponentially with the addition of more qubits.

In [ ]:
qmr = c2qa.QumodeRegister(
    num_qumodes=1,
    num_qubits_per_qumode=4
)

The Bosonic extension of Qiskit's `QuantumCircuit` is the `CVCircuit`, which is used to represent gates represented as instructions applied sequentially to the `CVCircuit`.

In [ ]:
circuit = c2qa.CVCircuit(qmr)

## Photons in a cavity
Qumodes are commonly implemented using a cavity, which is like a box with mirrored walls. Tlhe reflective walls of the box give it the ability to trap photons inside, and so photons can be added and removed from the box. Quantum mechanics tells us that photons are not just particles, but also electromagnetic waves. In the empty space inside the cavity, Maxwell's equations can be used to show that each component of the electric field in the cavity satisfies the equation
$$
\vec E(x,y,z,t) = \vec E \operatorname{Re}(e^{i(k_x x+k_yy+k_zz - \omega t)})
$$
where $\vec E$ is a constant and $\omega^2 = c^2(k_x^2+k_y^2+k_z^2)$. Maxwell's equations also require that any transverse component of the field vanishes at the walls, which gives the following conditions on $k_x, k_y, k_z$:
\begin{align*}
E(x, y, z) &= E(x+n_xL_x, y, z) &\implies k_x &= \frac{2\pi n_x}{L_x}\\
E(x, y, z) &= E(x, y+n_yL_y, z) &\implies k_y &= \frac{2\pi n_y}{L_y}\\
E(x, y, z) &= E(x, y, z+n_zL_z) &\implies k_z &= \frac{2\pi n_z}{L_z}
\end{align*}
For some integers $n_x, n_y, n_z$. This gives a set of discrete solutions for the allowed frequencies of photons in the cavity:
$$
\omega_{n_x, n_y, n_z}^2 = 2\pi c\left(\frac{n_x}{L_x}+\frac{n_y}{L_y}+\frac{n_z}{L_z}\right)
$$
The cavity can only capture photons with frequencies at these specific increments. This is illustrated for a 1D cavity in the diagram below:

![A cavity can only capture photons with a discrete set of frequencies determined by the boundary conditions on the electric field](./figures/qumodes.png)

The energy of a single photon is related to its angular frequency by $\varepsilon = \omega \hbar$. Thus, each allowed frequency $\omega_{n_x, n_y, n_z}$, or "mode" of the cavity, corresponds to an energy $\hbar\omega_{n_x, n_y, n_z}N_{n_x, n_y, n_z}$, where $N_{n_x, n_y, n_z}$ is the number of photons in that particular mode. Each of these cavity modes is an example of a qumode. This is certainly not the only way to represent qumodes. For further information, the connection between qumodes and harmonic oscillators is explored in `operations.ipynb`.

### Initializing Fock states

A Fock state is a state of the qumode with a definite number of photons. The `cv_initialize` method is used to initialize the qumode to different states. The following piece of code initializes the qumode to Fock state $\ket{0}$ (Refering back to the introduction, the number refers to the number of photons corresponding to that state). The first argument refers to which Fock state you want to initialize, and the second argument refers to which qumode you want to use. The Fock number cannot be higher than $\ket{2^k-1}$ for this qumode, where $k$ is the number of qubits per qumode.

In [ ]:
circuit.cv_initialize(0, qmr[0])

The method `cv\_initialize` can also create a superposition of Fock states. Instead of a value for the first argument, the method can take a list of the coefficients for each Fock state, with the $i^{th}$ coefficient corresponding to Fock state $\ket{i}$. The following piece of code initilizes the qumode to $\frac{\ket{0}+\ket{1}}{\sqrt{2}}$.

In [ ]:
circuit.cv_initialize([1/np.sqrt(2),1/np.sqrt(2)], qmr[0])

## How do qubits and bosonic modes interact?
At the heart of quantum computing with bosonic modes is the ability to make a qubit and a qumode interact. Just like both single- and two-qubit gates are necessary for universal control of a qubit, both Gaussian and non-Gaussian gates are necessary to control a qumode. For qubits that are not truly two-level systems, an anharmonic potential is necessary to isolate the first two energy levels. Non-Gaussian gates on a qumode are only possible because of the anharmonicity introduced by the qubit. The following shows what is meant by "anharmonicity." The figure below shows the cosine potential and first five energy levels of a transmon qubit vs the quadratic potential and first five energy levels of a qumode. The difference between the first two energy levels is chosen to be the same on both. The quadratic potential of the qumode leads to evenly spaced energy levels, whereas the levels of the transmon are not evenly spaced. This plot was produced using [scQubits](https://scqubits.readthedocs.io/en/latest/).

![Energy levels of a transmon compared to an oscillator](./figures/tmon.svg)

The physicist's prototype for describing qubit-qumode coupling is cavity quantum electrodynamics, which at its most basic level consists of a two-level atom embedded within a cavity. This is illustrated in the figure below, reproduced from [Wiring up Quantum Systems](https://www.nature.com/articles/451664a) by Shoelkopf et al.

![Diagram of a prototypical cQED system](./figures/cQED.png)

The atom interacts with the cavity by emitting a photon at the frequency of one of the modes of the cavity, which is reflected by the cavity and reabsorbed by the atom. This shifts the energy levels of the combined system so that they are not evenly spaced, illustrated in the following diagram reproduced from [Blais et al.](https://journals.aps.org/rmp/abstract/10.1103/RevModPhys.93.025005)


![Energy levels of the Jaynes-Cummings Hamiltonina](./figures/JCshift.png)

The state of the qubit is labeled as $|g\rangle$ (ground state) or $|e\rangle$ (first excited state), and the Fock states of the qumode are labeled numerically. The gray bars show the energy levels of the combined qumode-qubit system without considering the coupling, which depend only on the number of quanta, or particles, in the system. The coupling of the atom to a single mode of the cavity lifts this degeneracy, allowing for gate on the cavity which are conditioned on the state of the qubit.

## Combining Qumodes and Qubits
The following piece of code demostrates how the `QumodeRegister` can have multiple qumodes, qubits, and classical registers. In cQED, qubits are the main point of interaction with qumodes. Qumodes can have mich higher quality factors than the qubits they are coupled to, making them ideal for storing information with a long lifetime. Gates on the qumode can be performed through the qubit. In addition, the state of the qubit is dependent on the state of the cavity, and so only the state of the qubit needs to be read out. 

In [6]:
qmr = c2qa.QumodeRegister(
    num_qumodes=1,
    num_qubits_per_qumode=4
)

qr = qiskit.QuantumRegister(
    size=1
)

cr = qiskit.ClassicalRegister(
    size = 1
)

circuit = c2qa.CVCircuit(qmr, qr, cr)